In [36]:
#Author: Muhammad Azizi Bin Mohd Ariffin
#Email: mazizi@fskm.uitm.edu.my
#Description: Data pre-processing for deep learning traffic classification

import pyshark
import pandas as pd
import torch
import numpy as np
from sklearn import preprocessing
import nest_asyncio
nest_asyncio.apply()


In [37]:
import warnings
warnings.filterwarnings("ignore")

In [38]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

!nvidia-smi
#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

Tue Jul  2 13:35:17 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.86                 Driver Version: 551.86         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   50C    P8              4W /  123W |    1205MiB /   6141MiB |     24%      Default |
|                                         |                        |                  N/A |
+---------------------------

In [39]:
inputPath = "C:\\Users\\afif\\Documents\\Master\\Code\\Dataset\\NonVPN-PCAPs-03\\"
outputPath = "C:\\Users\\afif\\Documents\\Master\\Code\\Dataset\\NonVPN-PCAPs-01-processed\\"
fileName = "scpUp3"
fileExtension = ".pcap"
label = 'scp'

In [52]:
#c = pyshark.FileCapture("D:\\dataset\\ISCX-VPN2016\\CompletePCAPs\\aim_chat_3a.pcap", use_json=True, include_raw=True)
input = inputPath + fileName + fileExtension
c = pyshark.FileCapture(input, use_json=True, include_raw=True)
print("Done Read PCAP")
print(c[0])

Done Read PCAP
Packet (Length: 77)
Layer ETH
:	Destination: e8:e7:32:3c:65:50
	Address: e8:e7:32:3c:65:50
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Source: 98:90:96:a0:58:2f
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Type: IPv4 (0x0800)
	Address: 98:90:96:a0:58:2f
Layer IP
:	0100 .... = Version: 4
	.... 0101 = Header Length: 20 bytes (5)
	Differentiated Services Field: 0x00 (DSCP: CS0, ECN: Not-ECT)
	0000 00.. = Differentiated Services Codepoint: Default (0)
	.... ..00 = Explicit Congestion Notification: Not ECN-Capable Transport (0)
	Total Length: 63
	Identification: 0xf419 (62489)
	010. .... = Flags: 0x2, Don't fragment
	0... .... = Reserved bit: Not set
	.1.. .... = Don't fragment: Set
	..0. .... = More fragments: Not set
	...0 0000 0000 0000 = Fragment Offset: 0


In [41]:
countDf = 0
df = pd.DataFrame()
while countDf <= 1479 :
    index = 'B' + str(countDf)
    df[index] = np.nan
    countDf += 1

#print(df)

def padding(byteValue) :
    byteValue = byteValue.ljust(2960, '0')
    return byteValue

def truncate(byteValue) :
    byteValue = byteValue[0:2960]
    return byteValue

def processPacket(byteValue) :

    # Process to truncate or padding
    byteLen = len(byteValue)
    if byteLen > 2960 :
        byteValue = truncate(byteValue)
    elif byteLen < 2960 :
        byteValue = padding(byteValue)
        
    n = 2
    byteList = []

    #convert the packet from hexadecimal to decimal format
    for i in range(0, len(byteValue), n):
        temp = byteValue[i:i+n]
        base16INT = int(temp, 16)
        byteList.append(base16INT)
    return byteList

count = 0
countIndex = 0
n = 2
counttls = 0
countdns = 0
counttcp = 0
countudp = 0
counthttp = 0
countmdns = 0
countdtls = 0
countstun = 0
countgquic = 0
countrtcp = 0
checkString = ""

In [42]:


for x in c :
    print(count)
    if 'TLS' in c[count] :
        #print('TLS')
        counttls += 1
        try:
            byteValue = c[count].ip_raw.value + c[count].tcp_raw.value + c[count].tls_raw.value
        except:
            byteValue = c[count].ip_raw.value + c[count].tcp_raw.value + c[count].tls_raw.value[0]
        #print(byteValue)
        #print(len(byteValue))
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
        #break
    elif 'HTTP' in c[count] :
        counthttp += 1
        try : 
            byteValue = c[count].ip_raw.value + c[count].tcp_raw.value + c[count].http_raw.value + c[count].data_raw.value
        except :
            byteValue = c[count].ip_raw.value + c[count].tcp_raw.value + c[count].http_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    elif 'MDNS' in c[count] :
        countmdns += 1
        try :
            byteValue = c[count].ipv6_raw.value + c[count].udp_raw.value + c[count].mdns_raw.value
        except :
            byteValue = c[count].ip_raw.value + c[count].udp_raw.value + c[count].mdns_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    elif 'DNS' in c[count] and 'UDP' in c[count] :
        #print('DNS')
        countdns += 1
        byteValue = c[count].ip_raw.value + c[count].udp_raw.value + c[count].dns_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    elif 'DTLS' in c[count] :
        countdtls +=1
        c[count].ip_raw.value + c[count].udp_raw.value + c[count].dtls_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    elif 'GQUIC' in c[count] :
        countgquic += 1
        c[count].ip_raw.value + c[count].udp_raw.value + c[count].gquic_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    elif 'RTCP' in c[count] :
        countrtcp += 1
        c[count].ip_raw.value + c[count].udp_raw.value + c[count].rtcp_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    elif 'STUN' in c[count] :
        countstun += 1
        try :
            c[count].ip_raw.value + c[count].udp_raw.value + c[count].stun_raw.value
        except :
            try :
                c[count].ip_raw.value + c[count].tcp_raw.value + c[count].stun_raw.value
            except :
                c[count].ip_raw.value + c[count].stun_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    elif 'TCP' in c[count] :
        #print('TCP')
        if 'DATA' in c[count] :
            counthttp += 1
            try :
                byteValue = c[count].ip_raw.value + c[count].tcp_raw.value + c[count].http_raw.value + c[count].data_raw.value
                counthttp += 1
            except :
                byteValue = c[count].ip_raw.value + c[count].tcp_raw.value + c[count].data_raw.value
                counttcp += 1
        else :
            counttcp += 1
            byteValue = c[count].ip_raw.value + c[count].tcp_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    elif 'UDP' in c[count] and 'DATA' in c[count] :
        #print('UDP')
        countudp += 1
        byteValue = c[count].ip_raw.value + c[count].udp_raw.value + c[count].data_raw.value
        df.loc[countIndex] = processPacket(byteValue)
        countIndex += 1
    else :
        checkString = str(c[count].layers)
        if 'RTCP' in checkString :
            c[count].ip_raw.value + c[count].udp_raw.value + c[count].rtcp_raw.value
            df.loc[countIndex] = processPacket(byteValue)
            countIndex += 1
            countrtcp += 1
            print('RTCP')
        else :
            print(c[count].highest_layer)  
    count += 1
df['label'] = label
#print(df)
print('Done')

0
1
2
3
4
5
6
7
8
9
10
11
DNS_RAW
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
Done


In [43]:
output = outputPath + fileName + fileExtension + '.csv'
df.to_csv(output, index=False)
print(output)

C:\Users\afif\Documents\Master\Code\Dataset\NonVPN-PCAPs-01-processed\scpUp3.pcap.csv


In [44]:
print('Total packets inspected:', count)
print('Total packets extracted:', counttls + countdns + counttcp + countudp + counthttp + countmdns + countdtls + countstun + countgquic + countrtcp)
print('Total TLS:', counttls)
print('Total DNS:', countdns)
print('Total TCP:', counttcp)
print('Total UDP:', countudp)
print('Total HTTP:', counthttp)
print('Total MDNS:', countmdns)
print('Total DTLS:', countdtls)
print('Total STUN:', countstun)
print('Total GQUIC:', countgquic)
print('Total RTCP:', countrtcp)

Total packets inspected: 144
Total packets extracted: 143
Total TLS: 0
Total DNS: 32
Total TCP: 111
Total UDP: 0
Total HTTP: 0
Total MDNS: 0
Total DTLS: 0
Total STUN: 0
Total GQUIC: 0
Total RTCP: 0
